In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

StatementMeta(, acdac01c-7307-4c28-99f9-5a3c19ccb230, 3, Finished, Available)

In [2]:
DeltaTable.createIfNotExists(spark)\
          .tableName("G_OrderPriority")\
          .addColumn("OP_ID",LongType())\
          .addColumn("Order_Priority",StringType())\
          .addColumn("Created_TS",TimestampType())\
          .addColumn("Modified_TS",TimestampType())\
          .execute()

StatementMeta(, acdac01c-7307-4c28-99f9-5a3c19ccb230, 4, Finished, Available)

In [4]:
dataframe=spark.read.table("TMDW_FinalProj.g_orderpriority")

StatementMeta(, acdac01c-7307-4c28-99f9-5a3c19ccb230, 6, Finished, Available)

In [6]:
Maximum_Date=dataframe.selectExpr("coalesce(max(Modified_TS),'1900-01-01')").first()[0]

StatementMeta(, acdac01c-7307-4c28-99f9-5a3c19ccb230, 8, Finished, Available)

In [7]:
Maximum_Date

StatementMeta(, acdac01c-7307-4c28-99f9-5a3c19ccb230, 9, Finished, Available)

'1900-01-01'

In [8]:
dataframe_bze=spark.read.table("TMDW_FinalProj.bronzesales")

StatementMeta(, acdac01c-7307-4c28-99f9-5a3c19ccb230, 10, Finished, Available)

In [10]:
dataframe_bmod=dataframe_bze.select("Order_Priority").where(col("Modified_TS")>Maximum_Date).drop_duplicates()

StatementMeta(, acdac01c-7307-4c28-99f9-5a3c19ccb230, 12, Finished, Available)

In [14]:
Maximum_ID=dataframe.selectExpr("coalesce(max(OP_ID),0)").first()[0]

dataframe_final=dataframe_bmod.withColumn("OP_ID",Maximum_ID+monotonically_increasing_id()+1)

StatementMeta(, acdac01c-7307-4c28-99f9-5a3c19ccb230, 16, Finished, Available)

In [15]:
dataframe_gdelta=DeltaTable.forPath(spark,"Tables/g_orderpriority")
dataframe_bze_table=dataframe_final

StatementMeta(, acdac01c-7307-4c28-99f9-5a3c19ccb230, 17, Finished, Available)

In [16]:
dataframe_gdelta.alias("gld")\
             .merge(\
               dataframe_bze_table.alias("bze"),\
                "gld.Order_Priority==Bze.Order_Priority"\
             )\
             .whenMatchedUpdate(\
                  set={
                    "gld.Modified_TS":current_timestamp()
                  }
             )\
             .whenNotMatchedInsert(\
                    values={
                         "gld.OP_ID":"Bze.OP_ID",
                     "gld.Order_Priority":"bze.Order_Priority",
                     "gld.Created_TS":current_timestamp(),
                     "gld.Modified_TS":current_timestamp()
                    }
             )\
             .execute()

StatementMeta(, acdac01c-7307-4c28-99f9-5a3c19ccb230, 18, Finished, Available)

In [18]:
%%sql
select * from g_orderpriority

StatementMeta(, acdac01c-7307-4c28-99f9-5a3c19ccb230, 20, Finished, Available)

<Spark SQL result set with 4 rows and 4 fields>